In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import folium
from IPython.display import display

In [2]:
# load data
station_df = pd.read_csv('stations - mapping.csv')
zip_df = pd.read_csv('zip codes - mapping.csv')

In [3]:
# turn into geo dataframe
station_df['station_geom'] = station_df['station_geom'].astype(str).apply(wkt.loads) 
station_gdf = gpd.GeoDataFrame(station_df, geometry = 'station_geom', crs = 'EPSG:4326')

zip_df['zip_code_geom'] = zip_df['zip_code_geom'].astype(str).apply(wkt.loads) 
zip_gdf = gpd.GeoDataFrame(zip_df, geometry = 'zip_code_geom', crs = 'EPSG:4326')
zip_gdf['zip_code'] = zip_gdf['zip_code'].astype(str)

In [4]:
station_gdf[station_gdf['station_complex_name'] == '1 Av']

,station_complex_name,station_geom,ridership_2013,ridership_2018,routes,ridership_perc_change
0,1 Av,POINT (-73.98163 40.73095),23467,20998,L,-10.5


### Mapping

In [ ]:
# create base map
map = folium.Map(location = [40.7128, -74.0060],
                 zoom_start = 11,
                 tiles = 'Cartodb Positron')

# choropleth layer
folium.Choropleth(geo_data = zip_gdf.to_json(),
                  data = zip_gdf,
                  columns = ['zip_code','pop_perc_change'],
                  key_on = 'feature.properties.zip_code',
                  fill_color = 'BuPu',
                  fill_opacity = 0.7,
                  line_opacity = 0.3,
                  legend_name = 'Population Percent Change'
                  ).add_to(map)

# add tooltop for choropleth layer
folium.GeoJson(zip_gdf,
               name = 'Zip Codes',
               tooltip = folium.GeoJsonTooltip(fields = ['zip_code','pop_perc_change','emp_perc_change'],
                                               aliases = ['Zip Code:','Population % Change (2013-2018):','Employees % Change (2013-2018):'],
                                               localize = True,
                                               sticky = False,
                                               lables = True,
                                               style = ('background-color: white; color: #333; font-size: 12px;')
                                               )
               ).add_to(map)

# add markers for subway stations
for station, row in station_gdf.iterrows():
    popup_text = f"""
    <b>Station/Complex Name: {row.get('station_complex_name')}</b><br>
    Routes: {row.get('routes')}</b><br>
    Ridership % Change (2013-2018): {row.get('ridership_perc_change')}</b><br>
    """

    folium.Marker(location = [row.station_geom.y, row.station_geom.x],
                  #radius = 2.5,
                  #color = 'red',
                  icon = folium.Icon(color = 'black', icon = 'map-marker'),
                  fill = True,
                  fill_opacity = 1.0,
                  popup = folium.Popup(popup_text, max_width = 250)
                  ).add_to(map)

In [6]:
# save map
map.save('NYC zip codes and subway stations map.html')

map